In [ ]:
from pathlib import Path
import random
import shutil

# ========= CONFIGURACIÓN BÁSICA ========= #

random.seed(42)

ROOT = Path("data")           # Carpeta donde están Horse/Deer/Cow
CLASSES = ["Cow", "Deer", "Horse"]
IMG_EXTS = {".jpg", ".jpeg", ".png", ".JPG", ".JPEG", ".PNG"}

# Proporciones de split
TRAIN_P = 0.70
VAL_P   = 0.20   # test será 1 - TRAIN_P - VAL_P

# Carpetas destino
OUT_RGB  = ROOT / "format_rgb"
OUT_T    = ROOT / "format_t"

# Crear estructura de carpetas YOLO (RGB y T)
for OUT in [OUT_RGB, OUT_T]:
    for split in ["train", "val", "test"]:
        (OUT / f"images/{split}").mkdir(parents=True, exist_ok=True)
        (OUT / f"labels/{split}").mkdir(parents=True, exist_ok=True)

# ========= 1. RECOLECTAR MUESTRAS RGB Y T POR SEPARADO ========= #

rgb_samples = []   # lista de (img_path, label_path)
t_samples   = []   # lista de (img_path, label_path) para térmicas

for cls in CLASSES:
    cls_dir = ROOT / cls
    if not cls_dir.exists():
        print(f"⚠ Carpeta no encontrada: {cls_dir}, la salto.")
        continue

    for p in cls_dir.iterdir():
        # sólo archivos de imagen
        if p.suffix in IMG_EXTS:
            label_path = p.with_suffix(".txt")
            if not label_path.exists():
                print(f"⚠ Sin label (txt) para {p.name}, la salto.")
                continue

            if p.stem.endswith("_R"):
                # TÉRMICA
                t_samples.append((p, label_path))
            else:
                # RGB
                rgb_samples.append((p, label_path))

print(f"✅ Total muestras RGB: {len(rgb_samples)}")
print(f"✅ Total muestras térmicas (_R): {len(t_samples)}")


# ========= 2. FUNCIÓN DE SPLIT Y COPIA ========= #

def split_and_copy(samples, out_root: Path, desc: str):
    random.shuffle(samples)
    n = len(samples)
    n_train = int(n * TRAIN_P)
    n_val   = int(n * VAL_P)
    n_test  = n - n_train - n_val

    train = samples[:n_train]
    val   = samples[n_train:n_train + n_val]
    test  = samples[n_train + n_val:]

    print(f"\n{desc} → total={n}, train={len(train)}, val={len(val)}, test={len(test)}")

    def copy_pair(img_path: Path, label_path: Path, split: str):
        dst_img = out_root / f"images/{split}" / img_path.name
        dst_lbl = out_root / f"labels/{split}" / label_path.name
        shutil.copy2(img_path, dst_img)
        shutil.copy2(label_path, dst_lbl)

    for img_path, lbl_path in train:
        copy_pair(img_path, lbl_path, "train")
    for img_path, lbl_path in val:
        copy_pair(img_path, lbl_path, "val")
    for img_path, lbl_path in test:
        copy_pair(img_path, lbl_path, "test")


# ========= 3. APLICAR A RGB Y T ========= #

split_and_copy(rgb_samples, OUT_RGB, "RGB")
split_and_copy(t_samples,   OUT_T,   "Térmicas")

print("\n🎉 Organización completa:")
print(f" - YOLO RGB → {OUT_RGB}")
print(f" - YOLO T   → {OUT_T}")

from pathlib import Path

# YAML para RGB
yaml_rgb_path = Path("data/format_rgb/wildlife_rgb.yaml")
yaml_rgb_content = """path: data/format_rgb

train: images/train
val: images/val
test: images/test

names:
  0: Cow
  1: Deer
  2: Horse
"""
yaml_rgb_path.write_text(yaml_rgb_content)
print(f"✅ Archivo RGB creado en: {yaml_rgb_path.resolve()}")


# YAML para Térmicas
yaml_t_path = Path("data/format_t/wildlife_t.yaml")
yaml_t_content = """path: data/format_t

train: images/train
val: images/val
test: images/test

names:
  0: Cow
  1: Deer
  2: Horse
"""
yaml_t_path.write_text(yaml_t_content)
print(f"✅ Archivo Térmico creado en: {yaml_t_path.resolve()}")
